# Patient Stratification

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from clusteval import clusteval
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib_venn import venn3


In [2]:
import sys
import logging

nblog = open("rf_prediction.log", "w")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [7]:
dir ='lisbon'
df = pd.read_csv('../data/'+dir+'/data.csv', encoding='utf-8')
patient_code = df['Code']
patient_group = df['Group']
df.drop(columns = ['Code','Group'], inplace = True) # drop unwanted columns
df

,IBP7,FETUA,TTHY,A2GL,FBLN2,FINC,NBL1,PLTP,CH3L1,C1QC,...,CAHD1,LV223,LYSC,LAMB2,IGD,SEM7A,G6PI,IBP4,CNTN2,CEMIP
0,0.000329,0.000852,0.02010,0.000549,0.000497,0.00268,0.000045,0.001050,0.002210,0.000509,...,0.000151,0.000009,0.000015,0.000214,0.000052,0.000572,0.000109,3.140000e-05,0.000852,0.000035
1,0.000264,0.000662,0.01560,0.000277,0.000500,0.00298,0.000050,0.001210,0.000824,0.000587,...,0.000161,0.000043,0.000032,0.000272,0.000041,0.000466,0.000043,5.360000e-06,0.000804,0.000030
2,0.000197,0.001740,0.01190,0.000438,0.000234,0.00158,0.000029,0.000962,0.000638,0.000330,...,0.000131,0.000016,0.000043,0.000099,0.000091,0.000413,0.000011,5.850000e-06,0.000560,0.000048
3,0.000267,0.001040,0.01970,0.000666,0.000438,0.00166,0.000026,0.000792,0.001120,0.000304,...,0.000112,0.000005,0.000026,0.000174,0.000039,0.000535,0.000015,3.600000e-06,0.000469,0.000024
4,0.000200,0.001080,0.01310,0.000693,0.000075,0.00204,0.000035,0.001300,0.001600,0.000443,...,0.000102,0.000008,0.000018,0.000115,0.000066,0.000382,0.000007,5.840000e-06,0.000692,0.000055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.000195,0.000431,0.01280,0.000336,0.000055,0.00396,0.000026,0.001600,0.002530,0.000520,...,0.000150,0.000008,0.000222,0.000228,0.000223,0.000925,0.000003,4.740000e-06,0.000837,0.000033
64,0.000101,0.000716,0.00615,0.000183,0.000392,0.00287,0.000021,0.000943,0.001140,0.000697,...,0.000248,0.000016,0.000091,0.000070,0.000094,0.000649,0.000005,7.930000e-06,0.000378,0.000073
65,0.000237,0.000839,0.01350,0.000483,0.000091,0.00258,0.000027,0.000728,0.000776,0.000607,...,0.000070,0.000061,0.000017,0.000192,0.000045,0.000792,0.000001,1.300000e-06,0.000560,0.000071
66,0.000159,0.000663,0.00757,0.000362,0.000077,0.00325,0.000024,0.001250,0.000887,0.000558,...,0.000144,0.000044,0.000009,0.000031,0.000053,0.000571,0.000017,3.000000e-05,0.000497,0.000028


**Feature selection RF** 

In [5]:
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]


def sens(y_true, y_pred): return tp(y_true, y_pred) / \
    (fn(y_true, y_pred) + tp(y_true, y_pred))


def spec(y_true, y_pred): return tn(y_true, y_pred) / \
    (fp(y_true, y_pred) + tn(y_true, y_pred))

sensitivity_scorer = make_scorer(sens)

# Specificity scorer
specificity_scorer = make_scorer(spec)

# AUC scorer
auc_scorer = make_scorer(roc_auc_score, needs_proba=True)

# accuracy scorer
accuracy_scorer = make_scorer(accuracy_score)


C:\Users\danis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [5]:
# get ranked features list
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(df, patient_group)

# Get feature importances
importances = rf.feature_importances_

# Create a DataFrame for feature importance
feature_importances = pd.DataFrame({'Feature': df.columns, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

feature_importances

,Feature,Importance
35,COMP,0.034341
6,NBL1,0.028366
81,ITIH4,0.026803
115,DCC,0.025565
37,APOA1,0.024333
...,...,...
362,LMAN2,0.000000
363,CADH6,0.000000
364,CADM2,0.000000
365,BGH3,0.000000


In [ ]:
results = []
# Get feature names sorted by importance
ranked_features = feature_importances['Feature'].tolist()

for k in range(1,len(df.columns)+1):
    print(k)   
    top_n_features = ranked_features[:k]

    param_grid = {
        'n_estimators': [10,50, 100, 200,300],
        'max_depth': [None, 10, 15,20,30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'class_weight' : [None, 'balanced'],
        'max_features' : ['sqrt', 'log2', None]}
    
    # Perform grid search with cross-validation
    grid_search = GridSearchCV(RandomForestClassifier(random_state = 42), param_grid, cv= LeaveOneOut(), scoring='accuracy', n_jobs=-1)
    grid_search.fit(df[top_n_features], patient_group)

    # Get the index of the best model (based on accuracy score)
    best_index = grid_search.best_index_
    
    # Store results
    results.append({
    'best_params': grid_search.best_params_,
    'best_accuracy': grid_search.best_score_,
    'best_std_accuracy' :  grid_search.cv_results_['std_test_score'][best_index],
    '#features': k,
    'features': top_n_features
    })
    print('Accuracy: ', grid_search.best_score_)

# Find the best classifier based on accuracy
best_result = max(results, key=lambda x: x['best_accuracy'])
best = f"Number of features: {best_result['#features']}\n" + "Features:" + str(best_result['features']) +"\n" + \
f"Best Params: {best_result['best_params']}" +"\n" + f"Best Accuracy: {best_result['best_accuracy']:.4f} +/- {best_result['best_std_accuracy']:.4f}" 


# Create the LaTeX table as a string
latex_table = f"""
\\begin{{table}}[htbp]
\\centering
\\caption{{Selected Features {dir}}}
\\begin{{tabular}}{{|l|l|}}
\\hline
\\textbf{{Classifier Parameters}} & {best_result['best_params']} \\\\
\\hline
\\textbf{{Number of Features}} & {best_result['#features']} \\\\
\\hline
\\textbf{{Feature List}} & {', '.join(best_result['features'])} \\\\
\\hline
\\textbf{{Best Accuracy}} & {best_result['best_accuracy']:.2f} $\\pm$ {best_result['best_std_accuracy']:.2f} \\\\
\\hline
\\end{{tabular}}
\\label{{tab:best_classifier}}
\\end{{table}}
"""
print(best)
#print(latex_table)

with open(f'selected_feautres_{dir}.txt', 'w') as f:
    f.write(latex_table)


**Results**

Lisbon

    Features:['NRP2', 'COMP', 'NBL1', 'FETUA', 'A1AG1', 'APOA1', 'PGS2', 'DCC', 'HRG', 'APOL1', 'FBLN2', 'ITIH4', 'LFNG', 'C1QC', 'COEA1', 'FHR2', 'HEP2', 'IBP7', 'CNTP2', 'TTHY', 'ISLR', 'APLP1', 'FRIL', 'LV657', 'NCAN', 'ALS', 'CO6', 'A2GL', 'SLIK4', 'RTN4R', 'GRIA4', 'RENR', 'HPRT', 'NPTXR', 'SPRC', 'CO3', 'CNTN1', 'ENPP2', '1433G', 'PTGDS', 'ALDOC', 'LDHA', 'GLU2B', 'MRC1', 'CD166']

Coimbra

    Features: ['APOH', 'HEMO', 'IBP7', 'RNAS1', 'TTHY', 'GLYLB', 'CIP4', 'KV113', 'CNTP2']

Both Cohorts:

    Intersection of features:['CNTP2', 'TTHY', 'IBP7']
    Features: ['FETUA', 'IBP7', 'ENPP2', 'GLYLB', 'NBL1', 'FBLN2', 'PGS2', 'CFAI', 'C1QC', 'CO3', 'AACT', 'MYO6', 'DCC', 'VASN', 'SIAE', 'SCG1', 'C1S', 'APOA4', '1433G', 'SCG3', 'APOA1', 'CD109', 'CATF', 'CNTN1', 'NID2', 'NRP2', 'FINC', 'APOH', 'COMP', 'FHR2', 'HEP2', 'MRC2']

**Clustering** 

In [8]:
def clustering(df_clust, patient_code):

    # Initialize clustering
    ce = clusteval(cluster='agglomerative', linkage='ward', evaluate='silhouette')

    # Fit
    results = ce.fit(df_clust)
    cluster_labels = results['labx']

    # Plot
    # Step 3: Reduce dimensionality for scatter plot (using PCA for 2D)
    tsne = TSNE(n_components=2)
    X_tsne = tsne.fit_transform(df_clust)

    # Step 4: Plot the scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=cluster_labels, cmap='viridis', s=50)

    # Step 5: Annotate points with labels (here, we're using the original cluster labels)
    for i in range(X_tsne.shape[0]):
        plt.annotate(patient_code[i], (X_tsne[i, 0], X_tsne[i, 1]), fontsize= 8, ha='center')

    ce.plot()
    ce.plot_silhouette(cmap = 'bwr')
    ce.dendrogram()

In [24]:
if dir == 'lisbon':
    features = ['NRP2', 'COMP', 'NBL1', 'FETUA', 'A1AG1', 'APOA1', 'PGS2', 'DCC', 'HRG', 'APOL1', 'FBLN2', 'ITIH4', 'LFNG', 'C1QC', 'COEA1', 'FHR2', 'HEP2', 'IBP7', 'CNTP2', 'TTHY', 'ISLR', 'APLP1', 'FRIL', 'LV657', 'NCAN', 'ALS', 'CO6', 'A2GL', 'SLIK4', 'RTN4R', 'GRIA4', 'RENR', 'HPRT', 'NPTXR', 'SPRC', 'CO3', 'CNTN1', 'ENPP2', '1433G', 'PTGDS', 'ALDOC', 'LDHA', 'GLU2B', 'MRC1', 'CD166']
elif dir == 'coimbra':
    features = ['APOH', 'HEMO', 'IBP7', 'RNAS1', 'TTHY', 'GLYLB', 'CIP4', 'KV113', 'CNTP2']
else:
    features = ['CNTP2', 'TTHY', 'IBP7']
df_clust = df[features]
df_clust = df_clust.loc[patient_group == 'MCI-AD'].values

patient_code_clust = list(patient_code.loc[patient_group == 'MCI-AD'])

#print(df_clust.apply(lambda x: x.str.encode('ascii', 'ignore').decode('ascii') if x.dtype == "object" else x))
scaler = MinMaxScaler()
df_clust = scaler.fit_transform(df_clust)
print(df_clust[24])
clustering(df_clust, patient_code_clust)

[clusteval] >INFO> Saving data in memory.
[clusteval] >INFO> Fit with method=[agglomerative], metric=[euclidean], linkage=[ward]
[clusteval] >INFO> Evaluate using silhouette.


[8.46365505e-03 4.40764331e-01 3.90372083e-01 5.81081081e-01
 9.48486800e-01 5.61440678e-01 2.08986660e-01 2.54028436e-02
 4.25019425e-01 4.90367167e-01 6.02664411e-02 4.32286024e-01
 9.13437014e-04 4.40243902e-01 6.59341714e-01 1.73710904e-01
 3.67378049e-01 5.05874223e-01 3.83524277e-01 5.45205479e-01
 5.64241046e-02 3.05469556e-01 9.64673913e-02 1.09109109e-01
 1.60054720e-01 2.63549416e-01 1.50000000e-01 6.47855530e-01
 1.90775681e-01 2.13071363e-01 1.45619364e-03 1.25602495e-01
 5.73007153e-02 2.06664022e-01 1.22846842e-02 6.51094027e-01
 2.83333333e-01 1.91946309e-01 2.15557020e-01 4.76793249e-01
 2.61317630e-01 4.89479131e-01 3.08357349e-01 1.33883249e-01
 1.31931166e-02]


[clusteval] >INFO:   0%|          | 0/23 [00:00<?, ?it/s]

UnicodeEncodeError: 'charmap' codec can't encode characters in position 25-34: character maps to <undefined>

**Comparing Selected features** 

In [ ]:
set1 = set(['NRP2', 'COMP', 'NBL1', 'FETUA', 'A1AG1', 'APOA1', 'PGS2', 'DCC', 'HRG', 'APOL1', 'FBLN2', 'ITIH4', 'LFNG', 'C1QC', 'COEA1', 'FHR2', 'HEP2', 'IBP7', 'CNTP2', 'TTHY', 'ISLR', 'APLP1', 'FRIL', 'LV657', 'NCAN', 'ALS', 'CO6', 'A2GL', 'SLIK4', 'RTN4R', 'GRIA4', 'RENR', 'HPRT', 'NPTXR', 'SPRC', 'CO3', 'CNTN1', 'ENPP2', '1433G', 'PTGDS', 'ALDOC', 'LDHA', 'GLU2B', 'MRC1', 'CD166'])
set2 = set(['APOH', 'HEMO', 'IBP7', 'RNAS1', 'TTHY', 'GLYLB', 'CIP4', 'KV113', 'CNTP2'])
set3 = set(['FETUA', 'IBP7', 'ENPP2', 'GLYLB', 'NBL1', 'FBLN2', 'PGS2', 'CFAI', 'C1QC', 'CO3', 'AACT', 'MYO6', 'DCC', 'VASN', 'SIAE', 'SCG1', 'C1S', 'APOA4', '1433G', 'SCG3', 'APOA1', 'CD109', 'CATF', 'CNTN1', 'NID2', 'NRP2', 'FINC', 'APOH', 'COMP', 'FHR2', 'HEP2', 'MRC2'])
venn3([set1, set2, set3], ('Lisbon', 'Coimbra', 'Lis+Coimbra'))
# Calculate intersections
intersection_12 = set1 & set2
intersection_13 = set1 & set3
intersection_23 = set2 & set3
intersection_123 = set1 & set2 & set3
# Print intersection results
print("Intersection of Lis and Coim:", intersection_12)
print("Intersection of Lis and Lis+Coim:", intersection_13)
print("Intersection of Coim and Lis+Coim:", intersection_23)
